# Collaborative Filtering
Le filtrage collaboratif filtre les informations en utilisant les interactions et les données recueillies par le système auprès d'autres utilisateurs. Il repose sur l'idée que les personnes qui se sont mises d'accord dans leur évaluation de certains éléments sont susceptibles de le faire à nouveau à l'avenir.

Les systèmes de filtrage collaboratif se concentrent sur la relation entre les utilisateurs et les éléments. La similarité des éléments est déterminée par la similarité des évaluations de ces éléments par les utilisateurs qui ont évalué les deux éléments.

![alt text](https://cdn-images-1.medium.com/max/500/1*QvhetbRjCr1vryTch_2HZQ.jpeg)

Les données sont issue du dataset [Kaggle - News Portal User Interactions by Globo.com](https://www.kaggle.com/gspmoreira/news-portal-user-interactions-by-globocom#clicks_sample.csv)

La méthode utilisée va prédire des évaluations en utilisant les similarités entre les utilisateurs.

# Chargement des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle
import os
import glob
import random
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline


In [ ]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

## Installation et chargement de la bibliothèque Surprise
[Surprise](https://surprise.readthedocs.io/en/stable) est une bibliothèque de scikit pour construire et analyser des systèmes de recommandation qui traitent des données de notation explicites.

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 9.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619400 sha256=93febabff838eda108c89800002be1d7c7ba161566eb6851a7fc8a3272e37188
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import dump

# Chargement du jeu de données

In [ ]:
!wget "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+9+-+Réalisez+une+application+mobile+de+recommandation+de+contenu/news-portal-user-interactions-by-globocom.zip" data.zip
!unzip -q news-portal-user-interactions-by-globocom.zip
!unzip -q clicks.zip

--2021-10-10 06:29:56--  https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+9+-+R%C3%A9alisez+une+application+mobile+de+recommandation+de+contenu/news-portal-user-interactions-by-globocom.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.90.251
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.90.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 376587710 (359M) [application/zip]
Saving to: ‘news-portal-user-interactions-by-globocom.zip’

news-portal-user-in 100%[===================>] 359.14M  88.4MB/s    in 4.1s    

2021-10-10 06:30:00 (88.5 MB/s) - ‘news-portal-user-interactions-by-globocom.zip’ saved [376587710/376587710]

--2021-10-10 06:30:00--  http://data.zip/
Resolving data.zip (data.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘data.zip’
FINISHED --2021-10-10 06:30:00--
Total wall clock time: 4.3s
Downloaded: 1 fil

In [ ]:
articles_metadata = pd.read_csv('./articles_metadata.csv')
articles_metadata['datetime'] = pd.to_datetime(articles_metadata['created_at_ts'] / 1000, unit='s')
print(f"Articles from {articles_metadata['datetime'].min()} to {articles_metadata['datetime'].max()}")
articles_metadata.head()

Articles from 2006-09-27 11:14:35 to 2018-03-13 12:12:30


,article_id,category_id,created_at_ts,publisher_id,words_count,datetime
0,0,0,1513144419000,0,168,2017-12-13 05:53:39
1,1,1,1405341936000,0,189,2014-07-14 12:45:36
2,2,1,1408667706000,0,250,2014-08-22 00:35:06
3,3,1,1408468313000,0,230,2014-08-19 17:11:53
4,4,1,1407071171000,0,162,2014-08-03 13:06:11


In [ ]:
# Concatenation des differents fichiers que constitue le jeu de données
all_files = glob.glob("clicks/*.csv")
data = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    data.append(df)

clicks = pd.concat(data, axis=0, ignore_index=True)

Ce jeu de données contient 10 champs

|Field|Description|
|:----|:----------|
|user_id|The user id|
|session_id|The session id|
|session_start|Timestamp of the first interaction of the session|
|session_size|Number of interactions of the session|
|click_article_id|Article id interacted by the user|
|click_timestamp|Timestamp of the interaction|
|click_environment|Id of the Environment: 1 - Facebook Instant Article, 2 - Mobile App, 3 - AMP (Accelerated Mobile Pages), 4 - Web|
|click_deviceGroup|Id of the Device Type: 1 - Tablet, 2 - TV, 3 - Empty, 4 - Mobile, 5 - Desktop|
|click_os|Id of the Operational System: 1 - Other, 2 - iOS, 3 - Android, 4 - Windows Phone, 5 - Windows Mobile, 6 - Windows, 7 - Mac OS X, 8 - Mac OS, 9 - Samsung, 10 - FireHbbTV, 11 - ATV OS X, 12 - tvOS, 13 - Chrome OS, 14 - Debian, 15 - Symbian OS, 16 - BlackBerry OS, 17 - Firefox OS, 18 - Android, 19 - Brew MP, 20 - Chromecast, 21 - webOS, 22 - Gentoo, 23 - Solaris|
|click_country|Id of the country|

In [ ]:
clicks.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,47947,1507109782148845,1507109782000,4,57779,1507110097797,4,1,17,1,21,1
1,47947,1507109782148845,1507109782000,4,233717,1507110137895,4,1,17,1,21,1
2,47947,1507109782148845,1507109782000,4,58242,1507110445738,4,1,17,1,21,1
3,47947,1507109782148845,1507109782000,4,203241,1507110475738,4,1,17,1,21,1
4,132797,1507109787257846,1507109787000,2,361264,1507109806160,4,1,17,1,27,1


# Préparation des données

## Sélection des champs nécessaires
On ne sélectionne que les champs qui seront utilisés pour créer la note (rating): 
 - user_id
 - click_article_id
 - click_timestamp
 - session_size



In [ ]:
df = pd.DataFrame(clicks, columns=['user_id','click_article_id','click_timestamp','session_size'])
df.head()

,user_id,click_article_id,click_timestamp,session_size
0,47947,57779,1507110097797,4
1,47947,233717,1507110137895,4
2,47947,58242,1507110445738,4
3,47947,203241,1507110475738,4
4,132797,361264,1507109806160,2


## Création du champ rating
Pour batir la note nous allons utiliser le nombre de clicks sur l'article par l'utilisateur auquel on divise le nombre d'articles vu au cours de la même session.

In [ ]:
# Nombre de clicks sur un article pour un utilisateur
click_count_by_article = df.groupby(['user_id', 'click_article_id'], as_index=False).agg(
    click_count = pd.NamedAgg(column='click_article_id',aggfunc='count')
)
df = pd.merge(df, click_count_by_article, on=['user_id', 'click_article_id'])

# Nombre de clicks d'un utilisateur
user_clicks = df.groupby(['user_id'], as_index=False).agg(
    user_clicks = pd.NamedAgg(column='click_article_id',aggfunc='count')
)
df = pd.merge(df, user_clicks, on=['user_id'])

# Nombre de clicks sur un article / taille de la session
df['base_rating'] =  df['click_count'] / df['session_size']

# (Nombre de clicks sur un article / taille de la session) * nombre total de clicks de l'utilisateur
# df['base_rating'] =  (df['click_count'] / df['session_size']) * df['user_clicks']

# Nombre de clicks sur un article * nombre total de clicks de l'utilisateur
# df['base_rating'] =  df['click_count'] * df['user_clicks']

# Nombre de clicks sur un article + taille de la session + nombre total de clicks de l'utilisateur
# df['base_rating'] =  df['click_count'] + df['session_size'] + df['user_clicks']

# Nombre de clicks sur un article
# df['base_rating'] =  df['click_count']

df['base_rating'] = df['base_rating'].astype(float)
df.head()


,user_id,click_article_id,click_timestamp,session_size,click_count,user_clicks,base_rating
0,47947,57779,1507110097797,4,1,22,0.25
1,47947,233717,1507110137895,4,1,22,0.25
2,47947,58242,1507110445738,4,1,22,0.25
3,47947,203241,1507110475738,4,1,22,0.25
4,47947,357838,1507119062025,2,1,22,0.50


## Transformation de la variable rating en variable catégorielles

La variable **rating** qui est une variable continue, va être transformée en une variable catégorielle qui prendra les valeur de 0, 1, 2 ou 3 en fonction des quartilles.


In [ ]:
df['rating'] = pd.qcut(df['base_rating'], 4, labels=[0,1,2,3])

In [ ]:
df.head()

,user_id,click_article_id,click_timestamp,session_size,click_count,user_clicks,base_rating,rating
0,47947,57779,1507110097797,4,1,22,0.25,0
1,47947,233717,1507110137895,4,1,22,0.25,0
2,47947,58242,1507110445738,4,1,22,0.25,0
3,47947,203241,1507110475738,4,1,22,0.25,0
4,47947,357838,1507119062025,2,1,22,0.50,2


### Distribution des articles par score

In [ ]:
# Distribution des articles par score
data = df['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution de {} articles/score'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Quantité'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

### Distibution des scores par articles

In [ ]:
# Distibution des scores par articles

data = df.groupby('click_article_id')['rating'].count().clip(upper=50)

trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
layout = go.Layout(title = 'Distribution des scores par articles (Limité à 50)',
                   xaxis = dict(title = 'Nombre de scores par livre'),
                   yaxis = dict(title = 'Quantité'),
                   bargap = 0.2)


fig = go.Figure(data=[trace], layout=layout)
iplot(fig)


### Distribution des scores par utilisateurs

In [ ]:
# Distribution des scores par utilisateurs
data = df.groupby('user_id')['rating'].count().clip(upper=50)


trace = go.Histogram(x = data.values,
                     name = 'Scores',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))

layout = go.Layout(title = 'Distribution des scores par utilisateurs (limité à 50)',
                   xaxis = dict(title = 'Score par utilisateur'),
                   yaxis = dict(title = 'Quantité'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)


# Réduction du jeu de données
La distributions des scores par article ou par utilisateur montre qu'audelà d'une certaine limite le nombre de score ne devient plus pertinent.

En conséquence nous limiterons à 20 scores pour les articles et à 30 scores pour un utilisateur.

In [ ]:
# Filtre par nombre de lectures d'un article
article_rating_count = 20
article_rating = df['click_article_id'].value_counts() > article_rating_count
article_rating = article_rating[article_rating].index.tolist()
print(f"{len(article_rating)} articles with more than {article_rating_count} read.")

# Filtre pas nombre d'articles lu par un utilisateur
user_rating_count = 30
user_rating = df['user_id'].value_counts() > user_rating_count
user_rating = user_rating[user_rating].index.tolist()
print(f"{len(user_rating)} users with more than {user_rating_count} articles red.")

data = df[(df['click_article_id'].isin(article_rating)) & (df['user_id'].isin(user_rating))]
data.head()

6737 articles with more than 20 read.
17463 users with more than 30 articles red.


,user_id,click_article_id,click_timestamp,session_size,click_count,user_clicks,base_rating,rating
73,96521,160132,1507109816879,2,1,89,0.5,2
74,96521,58606,1507109846879,2,1,89,0.5,2
75,96521,293114,1507337389843,2,1,89,0.5,2
76,96521,293050,1507337419843,2,1,89,0.5,2
77,96521,124194,1507297202811,2,1,89,0.5,2


In [ ]:
data['rating'].value_counts()

0    515372
2    202465
1    170835
3     20676
Name: rating, dtype: int64

# Entrainement des algorithmes

 - L'algorithme **SVD** est équivalent à la factorisation matricielle probabiliste.
 - L'algorithme **SVDpp** est une extension de SVD qui prend en compte les évaluations implicites.
 - **SlopeOne** est une implémentation simple de l'algorithme SlopeOne.
 - L'algorithme **NormalPredictor** prédit un classement aléatoire en fonction de la distribution de l'ensemble d'apprentissage, qui est supposée être normale. Il s'agit de l'un des algorithmes les plus basiques qui n'effectuent pas beaucoup de travail.
 - **KNNBaseline** est un algorithme basique de filtrage collaboratif qui prend en compte une évaluation de base.
 - **KNNBasic** est un algorithme de filtrage collaboratif de base.
 - **KNNWithMeans** est un algorithme de filtrage collaboratif de base, qui prend en compte les évaluations moyennes de chaque utilisateur.
 - **KNNWithZScore** est un algorithme basique de filtrage collaboratif, prenant en compte la normalisation du z-score de chaque utilisateur.
 - L'algorithme **BaselineOnly** prédit l'estimation de base pour un utilisateur et un élément donnés.
 - **Coclustering** est un algorithme de filtrage collaboratif basé sur le co-clustering.





In [ ]:
# Surprise
reader = Reader(rating_scale=(0,3))
data_surprise = Dataset.load_from_df(data[['user_id','click_article_id','rating']], reader)

In [ ]:
benchmark = []
# Itération sur les algorithmes
for algorithm in [SVD(), SlopeOne(), NormalPredictor(), KNNBaseline(), KNNBasic()]:   
  # Cross validation
  results = cross_validate(algorithm, data_surprise, measures=['RMSE'], cv=3, verbose=False)
    
  # Traitement des résultats
  tmp = pd.DataFrame.from_dict(results).mean(axis=0)
  tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
  benchmark.append(tmp)

surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


## Sélection du meilleur algorithme

In [ ]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,0.758271,97.904806,341.172992
SVD,0.784716,36.680229,4.101290
KNNBasic,0.791534,91.291766,289.794409
SlopeOne,0.795000,4.766682,19.400514
NormalPredictor,1.152602,1.330710,3.719871


L'algorithme **KNNBaseline** montre de bonnes performances, mais te temps de traitement est très long, nous avons choisi le modèle **SVD** qui arrive en deuxième position.
La décomposition en valeurs singulières (SVD), est un algorithme de factorisation de matrices rectangulaires.


# Optimisation

In [22]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection import PredefinedKFold

reader = Reader(rating_scale=(0,3))
data_surprise = Dataset.load_from_df(df[['user_id','click_article_id','rating']], reader)


param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data_surprise)

print(f"Meilleur score RMSE {gs.best_score['rmse']}")
print(f"Meilleurs paramètres à utiliser : {gs.best_params['rmse']}")


Meilleur score RMSE 0.8157368199093207
Meilleurs paramètres à utiliser : {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}


# Entrainement de l'algorithme

In [23]:
algo = gs.best_estimator['rmse']
algo.fit(data_surprise.build_full_trainset())

# Sauvegarde de l'algorithme

In [ ]:
model_filename = "./model.dump"

# Dump algorithm
dump.dump(model_filename, algo=algo)
print(model_filename)

# Prédictions
Exemple de prédiction avec l'utilisateur n° **5**.
On va sélectionner le dernier article d'une période de 8 jours, puis comparer avec les articles lu les 8 jours suivants. 

Cet utilisateur à un historique de lecture qui va du 01/10/2017 au 16/10/2017, il a lu 87 articles durant cette période.

In [25]:
user_id = 5
clicks['datetime'] = pd.to_datetime(clicks['click_timestamp'] / 1000, unit='s')
user_click = clicks[clicks['user_id'] == user_id].sort_values('click_timestamp', ascending=False)
print(f"Articles from {user_click['datetime'].min()} to {user_click['datetime'].max()}")
print(f"This user have read {user_click.shape[0]}")


Articles from 2017-10-01 03:01:24.884999990 to 2017-10-16 22:19:20.851999998
This user have read 87


Sélection des articles lu durant la  période du 01/10/2017 au 08/10/2017

L'utilisateur à lu 33 articles durant cette période

In [ ]:
# 8 days
ref_start_date = '2017-10-01'
ref_end_date = '2017-10-08'
mask = (user_click['datetime'] > ref_start_date) & (user_click['datetime'] <= ref_end_date)
ref_period = user_click.loc[mask]

print(f"This user had read {ref_period.shape[0]} articles during 8 days")

Sélection des articles lu durant la période du 09/10/2017 au 16/10/2017

L'utilisateur à lu 42 articles durant cette période

In [ ]:
# Get next 8 days articles
pred_start_date = '2017-10-09'
pred_end_date = '2017-10-16'
mask = (user_click['datetime'] > pred_start_date) & (user_click['datetime'] <= pred_end_date)
pred_period = user_click.loc[mask]

print(f"This user have read {pred_period.shape[0]} article during 8 days")

Le dernier article lu le 07/10/2017 par l'utilisateur à le n° 202763, cet article a été publié le 06/10/2017

In [28]:
last_article = ref_period['click_article_id'][:1].iloc[0]
article_date = articles_metadata[articles_metadata['article_id'] == last_article]['datetime'].iloc[0]
last_article_date = ref_period['datetime'][:1].iloc[0]
print(f"On {last_article_date} the user read his last article #{last_article}, the article was published on {article_date}")

On 2017-10-07 14:52:53.525000095 the user read his last article #202763, the article was published on 2017-10-06 22:00:40


On peut supposer que l'utilisateur à tendance à lire les articles parus dans la semaine. Nous allons sélectionner uniquement les articles publiés durant la semaine de référence (du 09/10/2017 au 16/10/2017).

In [29]:
articles_read_list = pred_period['click_article_id'].tolist()
pred_articles = articles_metadata[articles_metadata['article_id'].isin(articles_read_list)]
print(f"{pred_articles.shape[0]} articles published during this period")


40 articles published during this period


Etant donné que seulement 40 articles ont été plubliés durant cete période et que l'utilisateur en à lu 42, il devient nécessaire t'étendre la recherche à un ensemble plus grand, nous utiliserond l'intervale de 2 semaines.

Cela représente 11637 articles.


In [30]:
# get articles published during this week
mask = (articles_metadata['datetime'] > ref_start_date) & (articles_metadata['datetime'] <= pred_end_date)
pred_period_articles = articles_metadata.loc[mask]
print(f"During this period {pred_period_articles.shape[0]} have been published")

During this period 11637 have been published


# Recommendations

In [31]:
score_list = []
for article_id in pred_period_articles['article_id'].tolist():                  
  rating = algo.predict(user_id, article_id)                  
  score_list.append([rating.uid, rating.iid,rating.est, round(rating.est)])
score_df = pd.DataFrame(score_list, columns=['user_id', 'article_id', 'raw', 'CF'])       
score_df = score_df.sort_values(by=['raw'], ascending=False)
score = score_df['article_id'][:5].tolist()
print(score)

[255354, 161178, 255068, 14392, 173772]


Le filtrage par contenu avait recommendé les articles : 363967, 363952, 363947, 363910, 363297.

## Évaluation

In [32]:
read = 0
not_read = 0

user_read = pred_period['click_article_id'].tolist()
for (idx, pred) in enumerate(score[:5]):
    if idx in user_read:
      read += 1
    else:
      not_read += 1  
    
print("-----")
print(f"Recommendations (already read): {read}")
print(f"Recommendations (not yet read): {not_read}")

-----
Recommendations (already read): 0
Recommendations (not yet read): 5


L'utilisation du "collaborative filtering", ne semble pas donner de bons résultats pour cet utilisateur. Un test A/B permettrait certainement de mieux vérifier la pertinence de cet algorithme.